In [122]:
import pandas as pd
from collections import Counter
import pprint
import logging, gensim, bz2
import pickle

In [123]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

In [84]:
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
brand_df=pd.read_excel('../../corpus/Coke vs Pepsi Twitter Data for NLP.xlsx')

In [16]:
brand_df.shape

(63000, 38)

In [6]:
brand_df.columns

Index([u'Sound Bite Text', u'Ratings and Scores', u'Title', u'Source Type',
       u'Post Type', u'Media Type', u'URL', u'Domain',
       u'Published Date (GMT+00:00) London', u'Author Gender', u'Author URL',
       u'Author Name', u'Author Handle', u'Author ID',
       u'Author Location - Country 1', u'Author Location - State/Province 1',
       u'Author Location - City 1', u'Author Location - Country 2',
       u'Author Location - State/Province 2', u'Author Location - City 2',
       u'Author Location - Other', u'No. of Followers/Daily Unique Visitors',
       u'Professions', u'Interests', u'Positive Objects', u'Negative Objects',
       u'Richness', u'Tags', u'Quoted Post', u'Quoted Author Name',
       u'Quoted Author Handle', u'Total Engagements', u'Post Comments',
       u'Post Likes', u'Post Shares', u'Product Name', u'Product Hierarchy',
       u'Rating'],
      dtype='object')

In [36]:
brand_df

,Sound Bite Text,Ratings and Scores,Title,Source Type,Post Type,Media Type,URL,Domain,Published Date (GMT+00:00) London,Author Gender,...,Quoted Post,Quoted Author Name,Quoted Author Handle,Total Engagements,Post Comments,Post Likes,Post Shares,Product Name,Product Hierarchy,Rating
0,"Mobile & Apps FILE - In this July 9, 2015, fil...",NaN,PepsiCo beats 3Q profit forecasts,Blogs,Original,No Media,http://www.mercedsunstar.com/news/business/art...,mercedsunstar.com,"Oct 4, 2017 11:19:00 AM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Mobile & Apps FILE - In this July 9, 2015, fil...",NaN,PepsiCo beats 3Q profit forecasts,Blogs,Original,No Media,http://www.mercedsunstar.com/news/business/art...,mercedsunstar.com,"Oct 4, 2017 11:19:00 AM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""When Patricia projects a vision of things to ...",NaN,"Stellar Scholars: When You Wish Upon a Star, H...",Blogs,Original,No Media,http://markets.financialcontent.com/stocks/new...,markets.financialcontent.com,"Oct 4, 2017 8:00:00 AM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RT @holyfuckordei: I shouldn’t be laughing thi...,NaN,NaN,Twitter,Retweets and Reblogs,Video,http://twitter.com/_Ravyyyy/statuses/914889802...,twitter.com,"Oct 2, 2017 5:28:21 PM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,From simply clicking the share buttons of posi...,NaN,10 Tips For Increasing Your Social Media Engag...,Blogs,Original,No Media,http://socialmediaweek.org/blog/2017/10/10-tip...,socialmediaweek.org,"Oct 3, 2017 6:16:14 PM",Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,But don’t go hashtag mad! Too many hashtags ca...,NaN,10 Tips For Increasing Your Social Media Engag...,Blogs,Original,No Media,http://socialmediaweek.org/blog/2017/10/10-tip...,socialmediaweek.org,"Oct 3, 2017 6:16:14 PM",Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RT @timelessbae: Selena's 1994 Coca-Cola photo...,NaN,NaN,Twitter,Retweets and Reblogs,Image,http://twitter.com/SamiDahling/statuses/914558...,twitter.com,"Oct 1, 2017 7:30:13 PM",Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RT @timelessbae: Selena's 1994 Coca-Cola photo...,NaN,NaN,Twitter,Retweets and Reblogs,Image,http://twitter.com/cynace24/statuses/914558103...,twitter.com,"Oct 1, 2017 7:30:18 PM",Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Not for nothing, but every time you blame the ...",NaN,NaN,Comments,Original,No Media,http://thehill.com/homenews/house/353550-giffo...,thehill.com,"Oct 3, 2017 12:48:54 AM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Not for nothing, but every time you blame the ...",NaN,NaN,Comments,Original,No Media,http://thehill.com/homenews/house/353550-giffo...,thehill.com,"Oct 3, 2017 12:48:54 AM",Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Important Columns

1. Title
2. Post Type
3. Media Type (Take only text)
4. Source Type
5. Domain
6. Richness

Important but blank

1. Interests
2. Positive Objects
3. Negative Objects

In [55]:
colnames_list = ['Sound Bite Text',"Title", "Post Type", "Media Type", "Source Type", "Domain", "Richness", "Interests", "Positive Objects", "Negative Objects"]

In [56]:
brand_df[colnames_list]

,Sound Bite Text,Title,Post Type,Media Type,Source Type,Domain,Richness,Interests,Positive Objects,Negative Objects
0,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN
1,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN
2,"""When Patricia projects a vision of things to ...","Stellar Scholars: When You Wish Upon a Star, H...",Original,No Media,Blogs,markets.financialcontent.com,4.0,NaN,NaN,NaN
3,RT @holyfuckordei: I shouldn’t be laughing thi...,NaN,Retweets and Reblogs,Video,Twitter,twitter.com,6.0,NaN,NaN,NaN
4,From simply clicking the share buttons of posi...,10 Tips For Increasing Your Social Media Engag...,Original,No Media,Blogs,socialmediaweek.org,3.0,NaN,NaN,NaN
5,But don’t go hashtag mad! Too many hashtags ca...,10 Tips For Increasing Your Social Media Engag...,Original,No Media,Blogs,socialmediaweek.org,1.0,NaN,Coca-Cola,NaN
6,RT @timelessbae: Selena's 1994 Coca-Cola photo...,NaN,Retweets and Reblogs,Image,Twitter,twitter.com,2.0,Music,NaN,NaN
7,RT @timelessbae: Selena's 1994 Coca-Cola photo...,NaN,Retweets and Reblogs,Image,Twitter,twitter.com,2.0,NaN,NaN,NaN
8,"Not for nothing, but every time you blame the ...",NaN,Original,No Media,Comments,thehill.com,5.0,NaN,Coca-Cola,"television industry, society"
9,"Not for nothing, but every time you blame the ...",NaN,Original,No Media,Comments,thehill.com,4.0,NaN,"gun, It","Wayne La Pierre, fault"


In [7]:
def get_most_common(df, colname):
    count_dict = Counter(df[colname])
    return count_dict.most_common(100)

In [8]:
pprint.pprint(get_most_common(brand_df, "Title"))

[(nan, 36712),
 (u'Coca-Cola increased its production of plastic bottles by a billion last year, says Greenpeace - Increase puts Coke\u2019s production at more than 110bn single-use plastic bottles a year',
  295),
 (u'The Coca-Cola Company and AB InBev Complete Transition of Majority Ownership in Coca-Cola Beverages Africa',
  184),
 (u'In push to get healthy, Pepsi may have pushed too hard in 3Q', 172),
 (u"Menu change outrages Domino's customers", 152),
 (u'You wanna find out some personal shit?', 140),
 (u'Century-old Coca-Cola distributer gulps new territory', 128),
 (u'Will Texans Still Love Topo Chico If Coca-Cola Owns It?', 122),
 (u"'I felt so f---ing stupid': Kendall Jenner breaks silence over Pepsi ad backlash",
  119),
 (u'Kendall Jenner Cries Over Pepsi Commercial [VIDEO]', 111),
 (u'PepsiCo beats 3Q profit forecasts', 92),
 (u'Coca-Cola increased its production of plastic bottles by a billion last year, says Greenpeace',
  90),
 (u'Trouble at Coca-Cola', 84),
 (u'Coca Col

In [13]:
pprint.pprint(get_most_common(brand_df, "Post Type"))

[(u'Original', 40474),
 (u'Retweets and Reblogs', 15246),
 (u'Replies and Comments', 7273),
 (nan, 7)]


In [15]:
pprint.pprint(get_most_common(brand_df, "Media Type"))

[(u'No Media', 32201),
 (u'Image', 13423),
 (u'Link', 10371),
 (u'Video', 4940),
 (u'Image; Link', 990),
 (u'Link; Image', 725),
 (u'Video; Link', 115),
 (u'Video; Image', 86),
 (u'Link; Video', 69),
 (u'Image; Video', 36),
 (u'Link; Audio Files', 23),
 (u'Audio Files', 11),
 (nan, 7),
 (u'Audio Files; Link', 3)]


In [23]:
pprint.pprint(get_most_common(brand_df, "Author Location - Country 1"))

[(nan, 35507),
 (u'United States', 18119),
 (u'United Kingdom', 2760),
 (u'Australia', 1139),
 (u'Canada', 842),
 (u'France', 432),
 (u'India', 396),
 (u'New Zealand', 325),
 (u'Germany', 323),
 (u'Nigeria', 298),
 (u'South Africa', 251),
 (u'Ireland', 136),
 (u'Singapore', 125),
 (u'China', 104),
 (u'Netherlands', 103),
 (u'Mexico', 100),
 (u'Tuvalu', 98),
 (u'Philippines', 95),
 (u'Spain', 94),
 (u'Malaysia', 94),
 (u'Italy', 87),
 (u'Brazil', 84),
 (u'Pakistan', 66),
 (u'United Arab Emirates', 62),
 (u'Zimbabwe', 62),
 (u'Kenya', 55),
 (u'Japan', 54),
 (u'Iceland', 52),
 (u'Indonesia', 48),
 (u'Argentina', 42),
 (u'Sweden', 42),
 (u'Czech Republic', 41),
 (u'Slovenia', 39),
 (u'Colombia', 38),
 (u'Thailand', 33),
 (u'Poland', 33),
 (u'Norway', 26),
 (u'Greece', 26),
 (u'Portugal', 23),
 (u'Egypt', 23),
 (u'Finland', 22),
 (u'Russia', 22),
 (u'Belgium', 22),
 (u'Estonia', 22),
 (u'Switzerland', 21),
 (u'Papua New Guinea', 21),
 (u'Ukraine', 20),
 (u'Vanuatu', 20),
 (u'Sri Lanka', 20)

In [24]:
pprint.pprint(get_most_common(brand_df, "Author Location - City 1"))

[(nan, 58447),
 (u'New York', 215),
 (u'London', 210),
 (u'Houston', 209),
 (u'Los Angeles', 135),
 (u'Chicago', 99),
 (u'Atlanta', 90),
 (u'Lagos', 73),
 (u'Austin', 61),
 (u'Toronto', 52),
 (u'Dallas', 49),
 (u'Paris', 47),
 (u'San Francisco', 45),
 (u'Manchester', 41),
 (u'Denver', 38),
 (u'Philadelphia', 32),
 (u'San Diego', 32),
 (u'Boston', 31),
 (u'Barcelona', 31),
 (u'Manhattan', 31),
 (u'Las Vegas', 30),
 (u'Melbourne', 30),
 (u'Seattle', 29),
 (u'San Antonio', 29),
 (u'Detroit', 28),
 (u'Orlando', 27),
 (u'Berlin', 25),
 (u'Mumbai', 23),
 (u'Sydney', 21),
 (u'Baltimore', 21),
 (u'Vancouver', 20),
 (u'Phoenix', 19),
 (u'St Louis', 18),
 (u'Miami', 18),
 (u'Pittsburgh', 18),
 (u'Curitiba', 16),
 (u'Louisville', 16),
 (u'Memphis', 16),
 (u'Birmingham', 16),
 (u'Glasgow', 16),
 (u'Delhi', 16),
 (u'Abuja', 16),
 (u'Cincinnati', 16),
 (u'Helsinki', 15),
 (u'Brooklyn', 15),
 (u'New Delhi', 14),
 (u'Hamburg', 14),
 (u'Rome', 14),
 (u'Cleveland', 14),
 (u'Liverpool', 13),
 (u'Charlott

In [25]:
pprint.pprint(get_most_common(brand_df, "Positive Objects"))

[(nan, 47855),
 (u'human flesh', 1302),
 (u'Pepsi', 510),
 (u'it', 460),
 (u'Coca-Cola', 296),
 (u'Coke', 198),
 (u'It', 179),
 (u'pepsi', 157),
 (u'Coca Cola', 134),
 (u'Diet Coke', 129),
 (u'sweet thing', 120),
 (u'10/5', 103),
 (u'fan', 102),
 (u'Pepsi Center WTC', 95),
 (u'coke', 95),
 (u'they', 76),
 (u'company', 76),
 (u'musical prowess', 69),
 (u'Topo Chico', 63),
 (u'free gala, Pepsi', 60),
 (u'Pepsi, Dew, Coke, Sprite', 53),
 (u"Uber's Board Reduces Kalanick's Clout", 48),
 (u'commercial', 48),
 (u'diet coke', 48),
 (u'#yum #yummy #coca cola #nomnomnom, #coca cola', 46),
 (u'coca cola', 45),
 (u'#coca cola', 45),
 (u'They', 43),
 (u'Pepsi max', 43),
 (u'offer, celebrity', 42),
 (u'RT, business', 41),
 (u'Kendall', 41),
 (u'new territory', 40),
 (u'sweet #brand, #sexvideo, #lund play #sexy video', 37),
 (u'Indra Nooyi', 36),
 (u'them', 30),
 (u'YDreams Global', 30),
 (u'best intention, Kendall', 30),
 (u'clarification', 28),
 (u'first pepsi smer', 28),
 (u'Keith, Citi Field, Co

In [42]:
pprint.pprint(get_most_common(brand_df, "Negative Objects"))

[(nan, 54436),
 (u'lipstick, cell, aborted fetus, placenta, dog fat', 1302),
 (u'Pepsi', 414),
 (u'it', 204),
 (u'Puerto Rico want, folk', 156),
 (u'they', 91),
 (u'NFL Sponsor, NFL', 83),
 (u'Kendall', 80),
 (u'coke', 73),
 (u'Coca-Cola', 71),
 (u'weaker-than-expected sale, it, weaker-than-expected sale', 71),
 (u'supremacy', 67),
 (u'Kendall Jenner', 62),
 (u'Coke', 62),
 (u'contract, Lakers star', 60),
 (u'pepsi', 58),
 (u'Coca-Cola HBC AG, share, Dimitris Lois', 57),
 (u'national anthem', 53),
 (u'Diet Coke', 49),
 (u'responsibility', 45),
 (u'Dimitris Lois', 42),
 (u'It', 40),
 (u'Coca-Cola, ketchup', 38),
 (u'girls', 37),
 (u'company', 35),
 (u'Coca Cola', 31),
 (u'fact', 30),
 (u'U.S. military, boycott, patriot', 29),
 (u'soda', 25),
 (u'people', 25),
 (u'sugar, coke, chemical', 25),
 (u'Coca-Cola, ketchup, mobile phone', 24),
 (u'Pepsi, public apology', 23),
 (u'model', 23),
 (u'tone-deaf commercial', 22),
 (u'brand', 22),
 (u'commercial, Pepsi, daughter, Caitlyn', 22),
 (u'com

In [35]:
pprint.pprint(get_most_common(brand_df, "Domain"))

[(u'twitter.com', 11091),
 (u'instagram.com', 10995),
 (u'tumblr.com', 7891),
 (u'facebook.com', 4037),
 (u'reddit.com', 583),
 (u'answers.yahoo.com', 368),
 (u'coca-colacompany.com', 316),
 (u'au.answers.yahoo.com', 287),
 (u'watchlistnews.com', 267),
 (u'ca.answers.yahoo.com', 258),
 (u'nz.answers.yahoo.com', 254),
 (u'zolmax.com', 251),
 (u'community.babycenter.com', 215),
 (u'uk.answers.yahoo.com', 209),
 (u'equitiesfocus.com', 208),
 (u'community.myfitnesspal.com', 201),
 (u'buzzfeed.com', 193),
 (u'4-traders.com', 182),
 (u'businessinsider.com', 181),
 (u'forums.sherdog.com', 175),
 (u'ledgergazette.com', 166),
 (u'tripadvisor.com', 160),
 (u'seekingalpha.com', 152),
 (u'breitbart.com', 149),
 (u'influenster.com', 146),
 (u'investorshub.advfn.com', 141),
 (u'highpointobserver.com', 133),
 (u'infowars.com', 127),
 (u'theguardian.com', 127),
 (u'boards.4chan.org', 122),
 (u'youtube.com', 120),
 (u'thestreet.com', 115),
 (u'24wrestling.com', 114),
 (u'dailypolitical.com', 110),
 (u'

In [41]:
pprint.pprint(get_most_common(brand_df, "Source Type"))

[(u'News', 11514),
 (u'Twitter', 11091),
 (u'Instagram', 10995),
 (u'Blogs', 10775),
 (u'Tumblr', 7891),
 (u'Forums', 5274),
 (u'Facebook', 4037),
 (u'Comments', 977),
 (u'Consumer Reviews', 317),
 (u'YouTube', 120),
 (nan, 7),
 (u'Other Social Networks', 2)]


In [38]:
pprint.pprint(get_most_common(brand_df, "Author Gender"))

[(u'Unknown', 44878), (u'Male', 9512), (u'Female', 8603), (nan, 7)]


id to index

In [40]:
pprint.pprint(get_most_common(brand_df, "Richness"))

[(1.0, 14315),
 (2.0, 13921),
 (0.0, 10646),
 (3.0, 10394),
 (4.0, 7154),
 (6.0, 3662),
 (5.0, 1439),
 (8.0, 1303),
 (7.0, 141),
 (9.0, 16),
 (10.0, 2),
 (nan, 1),
 (nan, 1),
 (nan, 1),
 (nan, 1),
 (nan, 1),
 (nan, 1),
 (nan, 1)]


In [46]:
pprint.pprint(get_most_common(brand_df, "Interests"))

[(nan, 60944),
 (u'Family', 215),
 (u'Religion', 179),
 (u'Sports', 145),
 (u'Food and Drink', 143),
 (u'Music', 141),
 (u'Politics', 137),
 (u'Gaming', 67),
 (u'Pets', 63),
 (u'Fashion', 58),
 (u'Religion; Family', 57),
 (u'Arts and Crafts', 55),
 (u'Technology', 51),
 (u'Photo and Video', 36),
 (u'Anime and Comics', 28),
 (u'Travel', 26),
 (u'Literature', 26),
 (u'Television', 25),
 (u'Religion; Politics', 20),
 (u'Green Living', 19),
 (u'Beauty', 17),
 (u'Health and Fitness', 16),
 (u'Sports; Family', 16),
 (u'Food and Drink; Family', 16),
 (u'Music; Arts and Crafts', 14),
 (u'Automotive', 13),
 (u'Family; Politics', 11),
 (u'Movies', 11),
 (u'Religion; Family; Politics', 10),
 (u'Travel; Family', 10),
 (u'Fashion; Arts and Crafts', 9),
 (u'Pets; Family', 9),
 (u'Music; Religion', 9),
 (u'Outdoors', 8),
 (u'Music; Food and Drink', 8),
 (u'Food and Drink; Politics', 8),
 (u'Music; Family', 7),
 (u'Food and Drink; Travel', 7),
 (u'Music; Gaming', 7),
 (u'Sports; Food and Drink', 7),
 

In [63]:
sel_df = brand_df.loc[brand_df["Media Type"].isin(["No Media"]) & brand_df["Post Type"].isin(["Original"]), colnames_list]

In [66]:
sel_df = sel_df.fillna("")

In [67]:
text_list  = list(sel_df["Sound Bite Text"].str.cat([sel_df["Title"],sel_df["Positive Objects"],sel_df["Negative Objects"]], sep = " "))

In [68]:
text_list

[u'Mobile & Apps FILE - In this July 9, 2015, file photo, Pepsi bottles are on display for sale at a supermarket in Haverhill, Mass. PepsiCo Inc. reports earnings, Wednesday, Oct. 4, 2017. Elise Amendola, File AP Photo FILE - In this July 9, 2015, file photo, Pepsi bottles are on display for sale at a supermarket in Haverhill, Mass. PepsiCo Inc. reports earnings, Wednesday, Oct. 4, 2017. Elise Amendola, File AP Photo PepsiCo beats 3Q profit forecasts The Associated Press October 04, 2017 3:19 AM PURCHASE, N.Y. PepsiCo Inc. is reporting third-quarter earnings of $2.14 billion. PepsiCo beats 3Q profit forecasts  ',
 u'Mobile & Apps FILE - In this July 9, 2015, file photo, Pepsi bottles are on display for sale at a supermarket in Haverhill, Mass. PepsiCo Inc. reports earnings, Wednesday, Oct. 4, 2017. Elise Amendola, File AP Photo FILE - In this July 9, 2015, file photo, Pepsi bottles are on display for sale at a supermarket in Haverhill, Mass. PepsiCo Inc. reports earnings, Wednesday, Oc

In [76]:
import nltk
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

def preprocess_text(text):
    #parse_text = BeautifulSoup(text).get_text()
    tokens = CountVectorizer(stop_words='english').build_tokenizer()(parse_text.lower())
    result = []
    
    
    for w in tokens:
        if w in stops:
            continue
        #lem_word = wordnet_lemmatizer.lemmatize(w)
        #stem_word = snowball_stemmer.stem(lem_word)
        result.append(w)
    return result

In [72]:
import pickle
pickle.dump(text_list, open("../../corpus/text_raw.pkl", "wb"))

In [107]:
tl = pickle.load(open("../../corpus/text_linewise.pkl"))

In [109]:
tl[0:10]

[[u'mobile apps file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass',
  u'pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass',
  u'pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo pepsico beats 3q profit forecasts the associated press october 04 2017 19 am purchase pepsico inc is reporting third quarter earnings of 14 billion',
  u'pepsico beats 3q profit forecasts'],
 [u'mobile apps file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass',
  u'pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass',
  u'pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo pepsico beats 3q profit f

In [110]:
clean_data = pickle.load(open("../../corpus/text_clean.pkl"))

In [115]:
clean_data = clean_data.replace(" <s_d_e_l> ", ". " )

In [116]:
corpus_doc = clean_data.split(" <d_d_e_l> ")

In [117]:
corpus_doc[0]

u'mobile apps file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass. pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo file in this july 2015 file photo pepsi bottles are on display for sale at supermarket in haverhill mass. pepsico inc reports earnings wednesday oct 2017 elise amendola file ap photo pepsico beats 3q profit forecasts the associated press october 04 2017 19 am purchase pepsico inc is reporting third quarter earnings of 14 billion. pepsico beats 3q profit forecasts'

In [118]:
corpus_doc_tokenized = map(preprocess_text,corpus_doc)

In [119]:
data = corpus_doc_tokenized
dictionary = gensim.corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]

In [124]:
dump(corpus, "../../corpus/corpus.pkl")
dump(dictionary, "../../corpus/dictionary.pkl")